In [18]:
import io
from pathlib import Path
import numpy as np
import pandas as pd
import soundfile as sf
import yaml
import tensorboard as tb
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

from IPython.display import Audio, HTML
from IPython.core.display import display

SAMPLE_RATE = 22050
CONFIGS_PATH = Path("configs")
LOGS_PATH = Path("logs")
LOGGED_AUDIOS_NUM = 10

In [19]:
def beep(data):
    display(Audio(data, rate=SAMPLE_RATE))

In [ ]:
pd.set_option('display.max_columns', None)
for config_path in CONFIGS_PATH.iterdir():
    if config_path.suffix != ".yaml":
        continue
    with open(config_path, "r") as stream:
        try:
            config = pd.json_normalize(yaml.safe_load(stream))
        except yaml.YAMLError as exc:
            print(exc)
    
    chp_logs_path = LOGS_PATH / f"{config['checkpoint_name'].values[0]}"
    if chp_logs_path.is_dir():
        display(config)
        ea = EventAccumulator(chp_logs_path)
        ea.Reload() 
        for audio_idx in range(LOGGED_AUDIOS_NUM):
            w_times, _, audio, _, _, _ = zip(*ea.Audio(f"Audio/Val/{audio_idx}"))
            data, _ = sf.read(io.BytesIO(audio[-1]))
            beep(data)

In [ ]:
# to convert to html
#!jupyter nbconvert report_for_logs.ipynb --no-input --to html --output report4logs